# PEFT

In [ ]:
# !pip install peft

In [ ]:
from peft import LoraConfig,TaskType

# 开启一个peft任务配置
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM,inference_mode=False,r=8,lora_alpha=32,lora_dropout=0.1)


In [6]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/mt0-large")

model.safetensors:  12%|#1        | 566M/4.92G [00:00<?, ?B/s]

In [7]:
from peft import get_peft_model

# 现有的模型+peft配置 = peft模型
model = get_peft_model(model,peft_config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 1,231,940,608 || trainable%: 0.1915


In [ ]:
# 可以使用 Transformers Trainer、Accelerate 或任何自定义 PyTorch 训练循环来训练模型
from transformers import TrainingArguments,Trainer
training_args = TrainingArguments(
    output_dir='./big-science/mt0-large-lora'
    ,learning_rate=1e-3
    ,per_device_train_batch_size=32
    ,per_device_eval_batch_size=32
    ,num_train_epochs=2
    ,weight_decay=0.01
    ,eval_strategy='epoch'
    ,save_strategy='epoch'
    ,load_best_model_at_end=True
)
trainer = Trainer(
    model = model
    ,args=training_args
    ,train_dataset=tokenized_datasets['train']
    ,eval_dataset=tokenized_datasets['test']
    ,tokenizer=tokenizer
    ,data_collator=data_collator
    ,compute_metrics=metrics,
)
trainer.train()

In [ ]:
# 保存模型
model.save_pretrained('./my_model')